<a href="https://colab.research.google.com/github/EngineerAbdullahIqbal/OpenAI-Agents-SDK/blob/main/00_OpenAI_Agents_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Agent Example

In [42]:
!pip install -Uq openai-agents # install openai-agents-sdk

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from google.colab import userdata

google_api_key = userdata.get('GEMINI_API_KEY')

In [6]:
from agents import AsyncOpenAI , OpenAIChatCompletionsModel
from agents.run import RunConfig
import asyncio

external_client = AsyncOpenAI(
    api_key=google_api_key,
     base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model='gemini-1.5-flash',
    openai_client=external_client,
)

run_config = RunConfig(
    model=model,
    tracing_disabled=True,
)

In [10]:
from agents import Agent , Runner

async def main():

  agent = Agent(
      name='Assistant',
      instructions='Answer The User Questions in All Languages.',
      model=model
  )

  result = await Runner.run(agent , 'mujhe salam kro' , run_config=run_config)

  #print Final_output
  print(result.final_output)


if __name__ == '__main__':
  asyncio.run(main())


Salam!  (Hello!)



# Multiple Agents with HandsOffs capability

## HandsOffs
-  Agents can Decide What Agent Do Particular Task


In [51]:
from agents import Agent

Junior_python_developer = Agent(
    name="junior Python Developer",
    handoff_description="Specialist agent for python programming",
    instructions="You provide simple solution of python code without type Hints. Base upon user Query.",
    model=model
)

senior_python_devloper = Agent(
    name="Senior Python Developer",
    handoff_description="Specialist agent for complex Python Problems",
    instructions="You provide help with Python Type Hints. Add Type Hints To The code.",
    model=model
)

In [54]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[Junior_python_developer , senior_python_devloper],
    model=model
)

Ways to Run an Agent
The Runner class provides three primary methods to run an agent, each tailored to specific use cases:

# 1. Runner.run()
### Type: Asynchronous (async).
Returns: A RunResult object once the agent completes its task.
Use Case: Perfect for scenarios where you want the agent to work in the background without halting other operations — common in web applications or multi-tasking environments.
How It Works: This method starts the agent asynchronously, allowing your code to continue running while the agent processes the input. You can use await to retrieve the result when it’s ready.

In [64]:
async def main():
    result = await Runner.run(triage_agent, "write a python function to add 2 numbers with type hints.")
    print(result.final_output)

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())

```python
def add_numbers(x: int, y: int) -> int:
  """Adds two numbers together.

  Args:
    x: The first number.
    y: The second number.

  Returns:
    The sum of x and y.
  """
  return x + y
```



# 2. Runner.run_sync()
### Type: Synchronous (sync).
Returns: A RunResult object once the agent finishes.
Use Case: Ideal for simple scripts or situations where you want to wait for the agent’s response before moving forward — no need to deal with async complexities.
How It Works: It runs the agent and blocks further execution until the task is complete. Internally, it leverages Runner.run() but simplifies the process by handling the async logic for you.


In [65]:
async def main():
    result = Runner.run_sync(triage_agent, "write a python function to add 2 numbers with type hints.")
    print(result.final_output)

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())

```python
def add_numbers(x: int, y: int) -> int:
  """Adds two numbers together.

  Args:
    x: The first number.
    y: The second number.

  Returns:
    The sum of x and y.
  """
  return x + y

```


# 3. Runner.run_streamed()
###Type: Asynchronous (async).
Returns: A RunResultStreaming object, providing real-time updates as the agent works.
Use Case: Great for applications where you want to show progress or partial results, like streaming text output to a user in a chat interface.
How It Works: The agent runs in streaming mode, delivering events (e.g., text chunks or tool call updates) as they occur, rather than waiting for the full task to finish.

In [63]:
from openai.types.responses import ResponseTextDeltaEvent

async def main():
    result = Runner.run_streamed(triage_agent, "write a python function to add 2 numbers with type hints.")
    async for event in result.stream_events():
      if event.type == 'raw_response_event' and isinstance(event.data , ResponseTextDeltaEvent):
        print(event.data.delta , end='' , flush=True)

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())

```python
def add_numbers(x: int, y: int) -> int:
  """Adds two numbers together.

  Args:
    

x: The first number.
    y: The second number.

  Returns:
    The sum of x and y.
  """
  return x + y
```


## Output Handling:
The LLM produces one of three types of outputs:
Final Output: A complete response (e.g., text or a structured result) with no further actions needed — the loop ends here.
Tool Calls: A request to use tools (e.g., fetching data or performing an action).
Handoff: A decision to pass the task to another agent.
### Loop Logic:
 - `If Final Output`: The loop stops, and the result is returned.

 - `If Tool Calls`:
The runner executes the tools, gathers their results, and adds them to the input.
The loop restarts with this updated input.
 - `If Handoff`:
The current agent switches to the new agent specified in the handoff.

The loop restarts with the new agent and adjusted input (if any).


`Termination:`
The loop keeps running until:
A final output is produced.
The maximum number of turns (max_turns) is exceeded, triggering a MaxTurnsExceeded exception to prevent infinite loops.
